## run this cell before starting spark cluster connection (corrects module error)

In [52]:
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [53]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [54]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
#import seaborn as sns

pd.set_option('display.max_columns', None)

### select the data

In [55]:
df_ps = spark.read.json("/user/sprenner/training_all_april.json")

In [56]:
df_ps.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_ps.columns]).limit(10).toPandas()

account  accountHash  clientState  clientStateHash  count  dataset  \
0        0            0            0                0   1624        0   

   datasetHash  datasetScope  datasetScopeHash  dataset_version  \
0            0             0                 0                0   

   dataset_versionHash  datatype  datatypeHash  eventType  eventTypeHash  \
0                    0         0             0          0              0   

   eventVersion  eventVersionHash  fileScope  filename  filenameHash  \
0             0                 0          0         0             0   

   filesize_double  hits  hostname  hostnameHash  hour  hourHash  id  \
0                0     0         0             0     0         0   0   

   id_proportion  int_percentage  join_key_april  join_key_lookup  \
0         197902          197902               0             1624   

   max_percentage  prod_step  prod_stepHash  project  projectHash  protocol  \
0          111420          0              0        0            0         0   

   protocolHash  remoteSite  remoteSiteHash  run_number  run_numberHash  \
0             0           0               0           0               0   

   scopeHash  stream_name  stream_nameHash  traceId  traceIdHash  traceIp  \
0          0            0                0        0            0        0   

   traceIpHash  traceTimeentryUnix  transferStart  uuid  uuidHash  
0            0                   0              0     0         0

account  accountHash  clientState  clientStateHash  count  dataset  \
0        0            0            0                0      0        0   

   datasetHash  datasetScope  datasetScopeHash  dataset_version  \
0            0             0                 0                0   

   dataset_versionHash  datatype  datatypeHash  eventType  eventTypeHash  \
0                    0         0             0          0              0   

   eventVersion  eventVersionHash  fileScope  filename  filenameHash  \
0             0                 0          0         0             0   

   filesize_double  hits  hostname  hostnameHash  hour  hourHash  id  \
0                0     0         0             0     0         0   0   

   id_proportion  int_percentage  join_key_april  join_key_lookup  \
0         197902               0               0             1624   

   max_percentage  prod_step  prod_stepHash  project  projectHash  protocol  \
0               0          0              0        0            0         0   

   protocolHash  remoteSite  remoteSiteHash  run_number  run_numberHash  \
0             0           0               0           0               0   

   scopeHash  stream_name  stream_nameHash  traceId  traceIdHash  traceIp  \
0          0            0                0        0            0        0   

   traceIpHash  traceTimeentryUnix  transferStart  uuid  uuidHash  
0            0                   0              0     0         0

### make buckets

In [59]:
from pyspark.ml.feature import Bucketizer

splits_hits = [-float("inf"), 2, float("inf")]
bucketizer = Bucketizer(splits=splits_hits, inputCol="hits", outputCol="bucketedHits")
bucketedData = bucketizer.transform(df_ps)
bucketedData.createOrReplaceTempView("bucket")

bucketedPercentage  count(1)
0                 0.0    527559
1                 1.0    249727

In [61]:
spark.sql("SELECT bucketedHits, count(*) \
            FROM bucket \
            GROUP BY  bucketedHits \
            ORDER BY 1 ASC").toPandas()

bucketedHits  count(1)
0           0.0    541829
1           1.0    235457

In [63]:
final_data = bucketedData.withColumn("class_bucket", when(final_data["bucketedHits"] == 0.0, "nocache").otherwise("cache"))

In [64]:
final_data.createOrReplaceTempView("final")

prefetch_bucket  count(1)
0      noprefetch    527559
1        prefetch    249727

In [65]:
spark.sql("SELECT class_bucket, count(*) \
            FROM final \
            GROUP BY  class_bucket \
            ORDER BY 1 ASC").toPandas()

class_bucket  count(1)
0        cache    235457
1      nocache    541829

In [66]:
final_data.describe().toPandas()

summary  account           accountHash         clientState  \
0   count   777286                777286              777286   
1    mean     None   1.935684591732567E8                None   
2  stddev     None  1.1361317241087065E9                None   
3     min  aagaard           -2127257360  CannotAuthenticate   
4     max   zzheng            2143592175      SourceNotFound   

        clientStateHash              count  \
0                777286             777286   
1   -9.74016245191591E8  3673.703860612439   
2  2.6212986830087823E8   7419.35514431962   
3           -1922357674                  0   
4            1269532611             199996   

                                             dataset           datasetHash  \
0                                             777286                777286   
1                                               None  -1.072936835947129E7   
2                                               None   1.258675129155887E9   
3  data10_hi.00169927.physics_bulk.ROOT.NTUP_HI.r...           -2146605558   
4  mc16_valid.423105.Pythia8EvtGen_A14NNPDF23LO_g...            2147407155   

  datasetScope      datasetScopeHash dataset_version   dataset_versionHash  \
0       777286                777286          777286                777286   
1         None  -7.552649492442358E8            None  1.3991617429934412E7   
2         None   8.314865501449597E8            None  1.3045708543551397E9   
3    data10_hi           -2057006107     c1130_m1847           -2146991447   
4   mc16_valid            2109319881      x557_m1947            2146405934   

  datatype          datatypeHash eventType eventTypeHash eventVersion  \
0   777286                777286    777286        777286       777286   
1     None   4.947060556405943E8      None    5.937808E7         None   
2     None  1.1700303292632792E9      None           0.0         None   
3      AOD           -2070622869  download      59378080       1.13.2   
4      log            2124512252  download      59378080       1.15.5   

       eventVersionHash   fileScope                            filename  \
0                777286      777286                              777286   
1  -1.992468944589716E9        None                                None   
2   3.073273964746096E8        None                                None   
3           -2044360925   data10_hi    AOD.05371635._000001.pool.root.1   
4            1261702107  mc16_valid  log.13787275._000072.job.log.tgz.1   

           filenameHash       filesize_double               hits  \
0                777286                777286             777286   
1   -248557.46481861244    6.11402028756827E8  2.010655022732945   
2  1.2402658149710774E9  1.2408771809945793E9   2.33759009262949   
3           -2147478094                 304.0                  1   
4            2147482617       1.1991446515E10                 40   

                   hostname           hostnameHash           hour  \
0                    777286                 777286         777286   
1                      None  -1.1804844919942595E8           None   
2                      None    1.228645222705116E9           None   
3    ReCaS-ui-01.cs.infn.it            -2133418459  2018-04-01 00   
4  xenia.nevis.columbia.edu             2129595856  2018-04-30 23   

                hourHash                 id       id_proportion  \
0                 777286             777286              579384   
1  -2.5176970566176414E7           388643.5   370425.3149948911   
2   1.2215909376898773E9  224383.2850061846  228391.37208069928   
3            -2138243719                  1                   2   
4             2144044799             777286              777286   

       int_percentage                                     join_key_april  \
0              777286                                             777286   
1  23.303134238877323                                               None   
2   28.43894253207948                                    

In [67]:
#final_data = final_data.withColumn("fileScopeHash", hash(final_data['fileScope']))

## Bucketing and Hashing End

In [68]:
final_data = final_data.orderBy("traceTimeentryUnix", ascending=True)

In [69]:
final_data.createOrReplaceTempView('final')
final_data = spark.sql("SELECT ROW_NUMBER() OVER (ORDER BY traceTimeentryUnix) as idx, * FROM final")

In [70]:
final_data.createOrReplaceTempView('final')
spark.sql("SELECT * FROM final WHERE idx <= 2585036").count()

621828

In [71]:
spark.sql("SELECT * FROM final WHERE idx > 2585036").count()

155458

In [72]:
from pyspark.ml.regression import LinearRegression

from pyspark.ml.evaluation import MulticlassClassificationEvaluator


#variante 1
#assembler = VectorAssembler(inputCols=["accountHash","clientStateHash","count","datasetHash","datasetScopeHash","dataset_versionHash",
#                                       "datatypeHash","eventTypeHash","eventVersionHash","filenameHash", "scopeHash","filesize_double","hostnameHash",
#                                       "hourHash","int_percentage","prod_stepHash","projectHash","protocolHash","remoteSiteHash","run_numberHash",
#                                       "stream_nameHash","traceIdHash","traceIpHash","traceTimeentryUnix","transferStart","uuidHash"], 
#                            outputCol="features")

#variante 2 (most important features according to Forward Selection)
assembler_hits = VectorAssembler(inputCols=["filesize_double","int_percentage","datatypeHash",
                                       "transferStart","hourHash","eventTypeHash"], 
                            outputCol="features_hits")

output = assembler_hits.transform(final_data)

output.createOrReplaceTempView('out')
train_data = spark.sql("SELECT * FROM out WHERE idx <= 2585036")
test_data = spark.sql("SELECT * FROM out WHERE idx > 2585036")

#train_data, test_data = output.randomSplit([.8,.2],seed=1234)

lr_hits = LinearRegression(labelCol='hits', featuresCol='features_hits', predictionCol="prediction_hits",maxIter=10, regParam=0.3, elasticNetParam=0.8)

model_hits = lr_hits.fit(train_data)
predictions_hits_percentage = model_hits.transform(output)
predictions_hits_percentage = predictions_hits_percentage.withColumn("linear_class_hits",when(predictions_hits_percentage.prediction_hits > 1, 1.0).otherwise(0.0))


evaluator = MulticlassClassificationEvaluator(labelCol="hits", predictionCol="linear_class_hits", metricName="accuracy")
accuracy = evaluator.evaluate(predictions_hits_percentage)
print("Test Error Hits = %g" % (1.0 - accuracy))

#Test Error hits forward bucketed: 0.386477

#Test Error hits forward regression: 0.410606

Test Error Percentage = 0.786856
Test Error Hits = 0.410606


In [73]:
predictions_hits_percentage = predictions_hits_percentage.withColumn("filesize", predictions_hits_percentage["filesize_double"].cast(IntegerType()))

In [74]:
final_output = predictions_hits_percentage.select('uuid', 'account', 'datasetScope', 'dataset', 'fileScope','filename', 'eventType', 'clientState', 
                                  from_unixtime(predictions_hits_percentage.traceTimeentryUnix, format='yyyy-MM-dd').alias('day'), 'traceTimeentryUnix', 
                                  'filesize', 'prediction_hits').sort(col("day").asc())

In [75]:
final_output.count()

155458

In [76]:
final_output.limit(10).toPandas()

uuid   account  datasetScope  \
0  8d160961f13a4effbc0a381a6518d5ca  tofitsch    mc15_14TeV   
1  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
2  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
3  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
4  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
5  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
6  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
7  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
8  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
9  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   

                                             dataset     fileScope  \
0  mc15_14TeV.167796.Sherpa_CT10_WtaunuMassiveCBP...    mc15_14TeV   
1  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
2  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
3  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
4  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
5  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
6  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
7  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
8  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
9  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   

                                    filename eventType         clientState  \
0   DAOD_TRUTH3.13422301._000001.pool.root.1  download                DONE   
1  DAOD_HIGG5D1.12819918._000580.pool.root.1  download  ServiceUnavailable   
2  DAOD_HIGG5D1.12819918._000581.pool.root.1  download  ServiceUnavailable   
3  DAOD_HIGG5D1.12819918._000581.pool.root.1  download  ServiceUnavailable   
4  DAOD_HIGG5D1.12819918._000580.pool.root.1  download  ServiceUnavailable   
5  DAOD_HIGG5D1.12819918._000580.pool.root.1  download  ServiceUnavailable   
6  DAOD_HIGG5D1.12819918._000582.pool.root.1  download  ServiceUnavailable   
7  DAOD_HIGG5D1.12819918._000582.pool.root.1  download  ServiceUnavailable   
8  DAOD_HIGG5D1.12819918._000582.pool.root.1  download  ServiceUnavailable   
9  DAOD_HIGG5D1.12819918._000582.pool.root.1  download  ServiceUnavailable   

          day  traceTimeentryUnix    filesize  prediction_hits  \
0  2018-04-22        1.524413e+09   236600961         0.996100   
1  2018-04-22        1.524413e+09  1327098000         2.410684   
2  2018-04-22        1.524413e+09  1379641362         2.432732   
3  2018-04-22        1.524413e+09  1379641362         2.432732   
4  2018-04-22        1.524413e+09  1327098000         2.410684   
5  2018-04-22        1.524413e+09  1327098000         2.410684   
6  2018-04-22        1.524413e+09  1398108348         2.440481   
7  2018-04-22        1.524413e+09  1398108348         2.440481   
8  2018-04-22        1.524413e+09  1398108348         2.440481   
9  2018-04-22        1.524413e+09  1398108348         2.440481   

   int_percentage  prediction_percentage  
0             100             111.302501  
1               0              33.866378  
2               0              33.486156  
3               0              33.486151  
4               0              33.866364  
5               0              33.866344  
6               0              33.352481  
7               0              33.352476  
8               0              33.352463  
9               0              33.352458

In [77]:
final_output = final_output.orderBy("traceTimeentryUnix", ascending=True)
final_output.limit(10).toPandas()

uuid   account  datasetScope  \
0  8d160961f13a4effbc0a381a6518d5ca  tofitsch    mc15_14TeV   
1  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
2  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
3  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
4  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
5  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
6  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
7  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
8  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
9  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   

                                             dataset     fileScope  \
0  mc15_14TeV.167796.Sherpa_CT10_WtaunuMassiveCBP...    mc15_14TeV   
1  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
2  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
3  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
4  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
5  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
6  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
7  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
8  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
9  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   

                                    filename eventType         clientState  \
0   DAOD_TRUTH3.13422301._000001.pool.root.1  download                DONE   
1  DAOD_HIGG5D1.12819918._000580.pool.root.1  download  ServiceUnavailable   
2  DAOD_HIGG5D1.12819918._000581.pool.root.1  download  ServiceUnavailable   
3  DAOD_HIGG5D1.12819918._000581.pool.root.1  download  ServiceUnavailable   
4  DAOD_HIGG5D1.12819918._000580.pool.root.1  download  ServiceUnavailable   
5  DAOD_HIGG5D1.12819918._000580.pool.root.1  download  ServiceUnavailable   
6  DAOD_HIGG5D1.12819918._000582.pool.root.1  download  ServiceUnavailable   
7  DAOD_HIGG5D1.12819918._000582.pool.root.1  download  ServiceUnavailable   
8  DAOD_HIGG5D1.12819918._000582.pool.root.1  download  ServiceUnavailable   
9  DAOD_HIGG5D1.12819918._000582.pool.root.1  download  ServiceUnavailable   

          day  traceTimeentryUnix    filesize  prediction_hits  \
0  2018-04-22        1.524413e+09   236600961         0.996100   
1  2018-04-22        1.524413e+09  1327098000         2.410684   
2  2018-04-22        1.524413e+09  1379641362         2.432732   
3  2018-04-22        1.524413e+09  1379641362         2.432732   
4  2018-04-22        1.524413e+09  1327098000         2.410684   
5  2018-04-22        1.524413e+09  1327098000         2.410684   
6  2018-04-22        1.524413e+09  1398108348         2.440481   
7  2018-04-22        1.524413e+09  1398108348         2.440481   
8  2018-04-22        1.524413e+09  1398108348         2.440481   
9  2018-04-22        1.524413e+09  1398108348         2.440481   

   int_percentage  prediction_percentage  
0             100             111.302501  
1               0              33.866378  
2               0              33.486156  
3               0              33.486151  
4               0              33.866364  
5               0              33.866344  
6               0              33.352481  
7               0              33.352476  
8               0              33.352463  
9               0              33.352458

## get complete amount of data downloaded in time window to calculate good cache size

In [78]:
final_output.createOrReplaceTempView('output')
spark.sql("SELECT SUM(filesize) FROM (SELECT DISTINCT fileScope, fileName, filesize FROM output)").toPandas()
#36398774222862

sum(filesize)
0  36398774222862

In [79]:
#final_output.write.csv('/user/sprenner/log_april_official_hourly_ordered_prefetching80_maxIter10_regParam03_elasticNetParam08_cache01.csv', sep='\t')